In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import time
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

A voir :
https://www.meteoblue.com/fr/meteo/historyclimate/weatherarchive/

In [4]:
def get_data_from_date(date):
    time.sleep(10)
    url = "https://prevision-meteo.ch/climat/horaire/paris-montsouris/" + date
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find('table', class_="table table-light-blue text-center").tbody
    data = []
    for tr in table.findAll("tr"):
        tds = tr.findAll("td")
        data.append([f"{date} {tds[0].text}", tds[1].text, tds[4].text, tds[5].text, tds[8].text, tds[9].text, tds[-1].text])
    return data

get_data_from_date("2021-11-30")

[['2021-11-30 23:00', '8.8', 'SO', '7.4', '83', '1012.2(-3.2/3h)', '--'],
 ['2021-11-30 22:00', '8.5', 'SO', '9.3', '85', '1013.4(-2.6/3h)', '--'],
 ['2021-11-30 21:00', '8.7', 'SO', '9.3', '85', '1014.5(-1.6/3h)', '--'],
 ['2021-11-30 20:00', '8.8', 'SO', '7.4', '84', '1015.4(-0.9/3h)', '--'],
 ['2021-11-30 19:00', '8.7', 'SO', '7.4', '84', '1016(-1.3/3h)', '--'],
 ['2021-11-30 18:00', '8.6', 'SO', '9.3', '84', '1016.1(-1.5/3h)', '--'],
 ['2021-11-30 17:00', '8.4', 'SO', '9.3', '85', '1016.3(-1.5/3h)', '--'],
 ['2021-11-30 16:00', '8.1', 'OSO', '7.4', '88', '1017.3(-0.5/3h)', '--'],
 ['2021-11-30 15:00', '8.1', 'OSO', '9.3', '89', '1017.6(-0.6/3h)', '--'],
 ['2021-11-30 14:00', '8.3', 'OSO', '9.3', '86', '1017.8(-1.1/3h)', '--'],
 ['2021-11-30 13:00', '8.0', 'OSO', '11.1', '85', '1017.9(-1.4/3h)', '--'],
 ['2021-11-30 12:00', '7.9', 'OSO', '7.4', '85', '1018.3(-1.0/3h)', '--'],
 ['2021-11-30 11:00', '7.1', 'OSO', '7.4', '86', '1019(-0.1/3h)', '--'],
 ['2021-11-30 10:00', '6.3', 'OSO',

In [5]:
from datetime import date, timedelta, datetime

startdate = date(2019, 1, 1)
enddate = date(2021, 12, 31)
days = [datetime.strftime(startdate + timedelta(days=i), "%Y-%m-%d") for i in range((enddate - startdate).days + 1)]

_startdate = date(2021, 1, 1)
_enddate = date(2021, 1, 10)
days_test = [datetime.strftime(_startdate + timedelta(days=i), "%Y-%m-%d") for i in range((_enddate - _startdate).days + 1)]

In [6]:
def get_all_data(_days):
    data = []
    for i, day in enumerate(_days):
        if i%100==0:
            print(i)
        try:
            data.append(get_data_from_date(day))
        except AttributeError as e:
            print(f"Error: {e} at attempt {i}")
            data.append([])
    return data

In [7]:
get_all_data(['2020-11-10', '2020-11-11'])

0
Error: 'NoneType' object has no attribute 'tbody' at attempt 0
Error: 'NoneType' object has no attribute 'tbody' at attempt 1


[[], []]

In [8]:
data = get_all_data(days)

0


KeyboardInterrupt: 

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [10]:
df = pd.DataFrame(flatten(data), columns=["date", "temp", "wind_dir", "wind_speed", "hum", "press", "weather_event"])
df

,date,temp,wind_dir,wind_speed,hum,press,weather_event
0,2019-01-01 23:00,8.0,NNE,11.1,79,1036.6(1.1/3h),--
1,2019-01-01 22:00,8.6,NNO,3.7,90,1036(0.6/3h),--
2,2019-01-01 21:00,8.6,N,5.6,90,1035.7(0.3/3h),--
3,2019-01-01 20:00,8.5,NNO,5.6,90,1035.5(0.3/3h),--
4,2019-01-01 19:00,8.3,NO,7.4,90,1035.4(0.8/3h),--
...,...,...,...,...,...,...,...
26010,2021-12-31 04:00,9.0,S,5.6,96,1021.5(-0.5/3h),--
26011,2021-12-31 03:00,9.5,S,11.1,96,1021.4(-0.8/3h),Ciel clair
26012,2021-12-31 02:00,9.6,SO,5.6,96,1022(-0.9/3h),Ciel clair
26013,2021-12-31 01:00,10.3,S,7.4,96,1022(-1.1/3h),Ciel clair


In [11]:
df.to_csv("./data/save-scrap-meteo.csv", sep=';', index=False)

# After scrapping

In [12]:
weather_events = {
    "--": "inconnu",
    "Il y a de la pluie faible et continue à la station au moment de l'observation": "pluie faible",
    "Du brouillard ou du brouillard glacé (plus correctement designé sous le nom de brume) est observé et la visibilité n'est ni inférieure à 5/8 de mille ni supérieure à 6 milles": "brouillard faible",
    "Il y a de la pluie faible et intermittente à la station au moment de l'observation": "pluie faible",
    "Ciel clair": "ciel clair",
    "Il y a de la pluie modérée et continue à la station au moment de l'observation": "pluie",
    "Il y a de la bruine et pluie mêlées d'intensité faible à la station au moment de l'observation": "pluie faible",
    "Il y a une bruine faible et continue à la station au moment de l'observation": "pluie faible",
    "Il y a une bruine faible et intermittente à la station au moment de l'observation": "pluie faible",
    "Il y a du brouillard ou du brouillard glacé à la station au moment de l'observation, la visibilité est inférieure à 5/8 de mille, le ciel est invisible et le brouillard n'a subi aucun changement d'intensité appréciable au cours de la dernière heure": "brouillard",
    "Il y a une bruine modérée et continue à la station au moment de l'observation": "pluie faible",
    "Il y a une faible chute de neige continue à la station au moment de l'observation": "neige faible",
    "Il y a de la pluie forte et continue à la station au moment de l'observation": "pluie forte",
    "Il y a de la pluie modérée et intermittente à la station au moment de l'observation": "pluie",
    "Il y a de la neige, accompagnée soit de bruine ou bruine verglaçante, soit de pluie ou pluie verglaçante à la station au moment de l'observation, et l'intensité de ces précipitations reste faible": "neige faible",
    "Il y a une bruine modérée et intermittente à la station au moment de l'observation": "pluie faible",
    "Il y a de la pluie forte mais intermittente à la station au moment de l'observation": "pluie forte",
    "Il y a du brouillard ou du brouillard glacé à la station au moment de l'observation, la visibilité est inférieure à 5/8 de mille, le ciel est invisible et le brouillard dépose du givre blanc": "brouillard fort",
    "Il y a de la neige, accompagnée soit de bruine ou bruine verglaçante, soit de pluie ou pluie verglaçante à la station au moment de l'observation, et l'intensité de ces précipitations est modérée ou forte": "neige",
    "Il y a une chute de neige modérée et continue à la station au moment de l'observation": "neige",
    "Il y a une faible chute de neige intermittente à la station au moment de l'observation": "neige faible",
    "Il y a une bruine forte mais intermittente à la station au moment de l'observation": "pluie",
    "Il y a une chute de neige forte et continue à la station au moment de l'observation": "neige forte",
    "Il y a des bancs de brouillard ou de brouillard glacé d'une épaisseur supérieure à 2m au moment de l'observation, et la visibilité dominante est réduite à moins de 5/8 de mille": "brouillard fort",
    "Il y a une bruine forte et continue à la station au moment de l'observation": "pluie forte",
    "Il y a de la bruine et pluie mêlées d'intensité modérée ou forte à la station au moment de l'observation": "pluie",
    "Il y a de la pluie verglaçante faible à la station au moment de l'observation": "pluie"
}

Units :
- temp : °C
- wind speed : km/h
- humidity : %
- pressure : hPA

In [14]:
df_saved = pd.read_csv("./data/save-scrap-meteo.csv", sep=';')
df_saved.weather_event = df_saved.weather_event.apply(lambda x: weather_events[x])
df_saved.press = df_saved.press.apply(lambda x: x.split("(")[0])
df_saved = df_saved.replace("--", np.nan)
df_saved["day"] = df_saved.date.apply(lambda x: x.split(" ")[0])
df_saved[["temp", "hum", "wind_speed", "press"]] = df_saved[["temp", "hum", "wind_speed", "press"]].astype("float32")
df_saved = df_saved[["date", "day", "temp", "wind_speed", "wind_dir", "hum", "press", "weather_event"]]
df_saved

,date,day,temp,wind_speed,wind_dir,hum,press,weather_event
0,2019-01-01 23:00,2019-01-01,8.0,11.1,NNE,79.0,1036.599976,inconnu
1,2019-01-01 22:00,2019-01-01,8.6,3.7,NNO,90.0,1036.000000,inconnu
2,2019-01-01 21:00,2019-01-01,8.6,5.6,N,90.0,1035.699951,inconnu
3,2019-01-01 20:00,2019-01-01,8.5,5.6,NNO,90.0,1035.500000,inconnu
4,2019-01-01 19:00,2019-01-01,8.3,7.4,NO,90.0,1035.400024,inconnu
...,...,...,...,...,...,...,...,...
26010,2021-12-31 04:00,2021-12-31,9.0,5.6,S,96.0,1021.500000,inconnu
26011,2021-12-31 03:00,2021-12-31,9.5,11.1,S,96.0,1021.400024,ciel clair
26012,2021-12-31 02:00,2021-12-31,9.6,5.6,SO,96.0,1022.000000,ciel clair
26013,2021-12-31 01:00,2021-12-31,10.3,7.4,S,96.0,1022.000000,ciel clair


In [15]:
df_saved.to_csv("./data/final/data-meteo.csv", sep=';', index=False)